依次执行下面的cells，通过可视化验证你的实现是否正确

In [ ]:
import torch
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import matplotlib.patches as mpatches

from exercises.tool import gen_dx_bx, denormalize_img, add_ego
from exercises.models import compile_model


In [ ]:
H=900
W=1600
resize_lim=(0.193, 0.225)
final_dim=(128, 352)
bot_pct_lim=(0.0, 0.22)
rot_lim=(-5.4, 5.4)
rand_flip=True

xbound=[-50.0, 50.0, 0.5]
ybound=[-50.0, 50.0, 0.5]
zbound=[-10.0, 10.0, 20.0]
dbound=[4.0, 45.0, 1.0]

grid_conf = {
    'xbound': xbound,
    'ybound': ybound,
    'zbound': zbound,
    'dbound': dbound,
}
cams = ['CAM_FRONT_LEFT', 'CAM_FRONT', 'CAM_FRONT_RIGHT',
    'CAM_BACK_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT']

data_aug_conf = {
                'resize_lim': resize_lim,
                'final_dim': final_dim,
                'rot_lim': rot_lim,
                'H': H, 'W': W,
                'rand_flip': rand_flip,
                'bot_pct_lim': bot_pct_lim,
                'cams': cams,
                'Ncams': 5,
            }

In [ ]:
# initialize lss model
model = compile_model(grid_conf, data_aug_conf, outC=1)

`model525000.pt`为我们提供的预训练好的BEV车辆分割模型权重，加载模型权重：

In [ ]:
model_path='model525000.pt'
print('loading', model_path)
model.load_state_dict(torch.load(model_path))

gpuid=0
device = torch.device('cpu') if gpuid < 0 else torch.device(f'cuda:{gpuid}')
model.to(device)
model.eval()

我们提供了一张图片，以.npy格式存储，`sample_imgs.npy`，以及

`model_parameters.pkl`: 包含该图片对应的模型入参，具体包括：
- rots：由相机坐标系->车身坐标系的旋转矩阵，rots = (bs, N, 3, 3)；
- trans：由相机坐标系->车身坐标系的平移矩阵，trans=(bs, N, 3)；
- intrinsic：相机内参，intrinsic = (bs, N, 3, 3)；
- post_rots：由图像增强引起的旋转矩阵，post_rots = (bs, N, 3, 3)；
- post_trans：由图像增强引起的平移矩阵，post_trans = (bs, N, 3)；

In [ ]:
# 加载.npy文件
loaded_array = np.load('sample_imgs.npy')

# 将NumPy数组转换回PyTorch张量
imgs = torch.from_numpy(loaded_array)
imgs = imgs.unsqueeze(0)

with open('model_parameters.pkl', 'rb') as f:
    loaded_parameters_dict = pickle.load(f)

# 从加载的字典中获取张量
rots = loaded_parameters_dict['rots'].unsqueeze(0)
trans = loaded_parameters_dict['trans'].unsqueeze(0)
intrins = loaded_parameters_dict['intrins'].unsqueeze(0)
post_rots = loaded_parameters_dict['post_rots'].unsqueeze(0)
post_trans = loaded_parameters_dict['post_trans'].unsqueeze(0)

对加载的图片进行可视化

In [ ]:
# visualize
%matplotlib inline
val = 0.01
final_dim=(128, 352)
fH, fW = final_dim

# fig = plt.figure(figsize=(3*fW*val, (1.5*fW + 2*fH)*val))
# gs = gridspec.GridSpec(3, 3, height_ratios=(1.5*fW, fH, fH))
fig = plt.figure(figsize=(3*fW*val, (2*fH)*val))
gs = gridspec.GridSpec(2, 3, height_ratios=(fH, fH))
gs.update(wspace=0.0, hspace=0.0, left=0.0, right=1.0, top=1.0, bottom=0.0)
for imgi, img in enumerate(imgs[0]):
    ax = plt.subplot(gs[imgi // 3, imgi % 3])
    showimg = denormalize_img(img)
    # flip the bottom images
    if imgi > 2:
        showimg = showimg.transpose(Image.FLIP_LEFT_RIGHT)
    ax.imshow(showimg)
    ax.axis('off')
    ax.annotate(cams[imgi].replace('_', ' '), (0.01, 0.92), xycoords='axes fraction')

plt.savefig("raw_image.jpg")
plt.show()


使用`get_geometry()`，将视锥点由图像坐标系向ego坐标系进行坐标转化，并可视化：

In [ ]:
geom = model.get_geometry(rots.to(device), trans.to(device), intrins.to(device), post_rots.to(device), post_trans.to(device))
print(geom.shape)

In [ ]:
# visualize
img_pts = geom.cpu()

fig = plt.figure(figsize=(8, 8))

for imgi, img in enumerate(imgs[0]):
    plt.plot(img_pts[0, imgi, :, :, :, 0].view(-1), img_pts[0, imgi, :, :, :, 1].view(-1), '.', label=cams[imgi].replace('_', ' '))

plt.savefig("geom.jpg")
plt.show()


调用`get_cam_feats`构建图像特征点云，再调用`voxel_pooling`构建BEV特征：

In [ ]:
x = model.get_cam_feats(imgs.to(device))
bev_feat = model.voxel_pooling(geom, x)

对得到的BEV特征可视化：

BEV特征的通道数为64，这里我们采取先对特征值求绝对值，再沿通道维度求平均的方法将BEV特征压缩

In [ ]:

bev_feat_np = bev_feat.detach().cpu().numpy()

# Take the absolute values
bev_feat_abs = np.abs(bev_feat_np)

# Calculate the mean across all channels
bev_feat_mean = np.mean(bev_feat_abs, axis=1)
bev_feat_mean = np.clip(bev_feat_mean,0,1.0)

# Visualize the result
plt.imshow(bev_feat_mean[0], cmap='jet')

plt.colorbar()
plt.axis('off')
plt.savefig("bev.jpg")
plt.show()


下面我们通过构建好的LSS模型，对输入数据进行前向传播，做car segmentation，得到最终的增强BEV特征：

In [ ]:

with torch.no_grad():
    out = model(imgs.to(device),
            rots.to(device),
            trans.to(device),
            intrins.to(device),
            post_rots.to(device),
            post_trans.to(device),
            )

对得到的BEV特征可视化，方法如前：

In [ ]:
# Take the absolute values
out_np = out.detach().cpu().numpy()

out_abs = np.abs(out_np)

# Calculate the mean across all channels
out_mean = np.mean(out_abs, axis=1)
# Visualize the result
plt.xlim((out.shape[3], 0))
plt.ylim((0, out.shape[3]))
plt.imshow(out_mean[0], cmap='viridis')

plt.colorbar()
plt.axis('off')
plt.savefig("out.jpg")
plt.show()


对输出进行 sigmoid 操作，模型的输出转为概率值，并将结果移动到 CPU 上

In [ ]:
out = out.sigmoid().cpu()

对模型预测的车辆分割结果进行可视化：

In [ ]:
# visualize
fig = plt.figure(figsize=(1.5*fW*val, (1.5*fW)*val))
dx, bx, _ = gen_dx_bx(grid_conf['xbound'], grid_conf['ybound'], grid_conf['zbound'])
dx, bx = dx[:2].numpy(), bx[:2].numpy()

plt.legend(handles=[
    mpatches.Patch(color=(0.0, 0.0, 1.0, 1.0), label='Output Vehicle Segmentation'),
    mpatches.Patch(color='#76b900', label='Ego Vehicle'),
], loc=(0.01, 0.86))
plt.imshow(out[0].squeeze(0), vmin=0, vmax=1, cmap='Blues')

plt.xlim((out.shape[3], 0))
plt.ylim((0, out.shape[3]))
add_ego(bx, dx)
plt.axis('off')

plt.savefig("car_seg.jpg")
plt.show()